In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import multivariate_normal, permutation
import pandas as pd
from pandas import DataFrame, Series

In [2]:
np.random.seed(20170921)

# tが０のデータを準備する
n0, mu0, variance0 = 20, [10,11], 20
data0 = multivariate_normal(mu0, np.eye(2)*variance0, n0)
df0 = DataFrame(data0, columns=['x1', 'x2'])
df0['t'] = 0

# tが1のデータを準備する
n1, mu1, variance1 = 20, [10,11], 20
data1 = multivariate_normal(mu1, np.eye(2)*variance1, n1)
df1 = DataFrame(data1, columns=['x1', 'x2'])
df1['t'] = 1

# tが1のものと0のものとを統合
df = pd.concat([df0, df1], ignore_index=True)
# 順番をランダムに並び替え
train_set = df.reindex(permutation(df.index)).reset_index(drop=True)

In [3]:
# pandasのデータフレームから入力値と正解値を別々の配列として取り出す
train_x = train_set[['x1', 'x2']].as_matrix()
train_t = train_set['t'].as_matrix().reshape([len(train_set), 1])

In [4]:
with tf.name_scope('X'):
    x = tf.placeholder(tf.float32, [None, 2])
with tf.name_scope('W'):
    w = tf.Variable(tf.zeros([2,1]))
with tf.name_scope('B'):
    b = tf.Variable(tf.zeros([1]))
with tf.name_scope('f'):
    f = tf.matmul(x, w) + b
with tf.name_scope('P'):
    p = tf.sigmoid(f)
with tf.name_scope('T'):
    t = tf.placeholder(tf.float32, [None, 1])
sess = tf.Session()

In [5]:
with tf.name_scope('loss'):
    loss = -tf.reduce_sum(t*tf.log(p) + (1-t)*tf.log(1-p))
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer().minimize(loss)

In [6]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.sign(p-0.5), tf.sign(t-0.5))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
# TensorBoardで追跡する変数を定義
with tf.name_scope('summary'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('loss', loss)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./logs', sess.graph)

In [8]:
sess.run(tf.global_variables_initializer())

i = 0
for _ in range(20000):
    i += 1
    __, summary = sess.run([train_step, merged], feed_dict={x: train_x, t: train_t})
    if i % 2000 == 0:
        loss_val,acc_val = sess.run([loss, accuracy], feed_dict={x: train_x, t: train_t})
        writer.add_summary(summary, _)
        print('step: %d, loss: %f, accuracy: %f' % (i, loss_val, acc_val))

step: 2000, loss: 25.293262, accuracy: 0.675000
step: 4000, loss: 25.082136, accuracy: 0.650000
step: 6000, loss: 25.079262, accuracy: 0.625000
step: 8000, loss: 25.079262, accuracy: 0.625000
step: 10000, loss: 25.079262, accuracy: 0.625000
step: 12000, loss: 25.079262, accuracy: 0.625000
step: 14000, loss: 25.079262, accuracy: 0.625000
step: 16000, loss: 25.079266, accuracy: 0.625000
step: 18000, loss: 25.079262, accuracy: 0.625000
step: 20000, loss: 25.079262, accuracy: 0.625000
